# 目录
## 1. 导入模块
## 2. `tf.function`的用法
  - ### 2.1 函数式用法
  - ### 2.2 `@tf.function`装饰器的用法
  
## 3. `tf.autograph` 的用法
  - ### 3.1 `tf.autograph.to_graph`,是`tf.function` 的低级 分装
  - ### 3.2 `tf.autograph.to_code`, 把函数变成图，再输出代码
  
## 4. 变量不能包含在`tf.function`内
## 5. `tf.function`签名 `input_signature`
## 6. `get_concrete_function`, 把变成图的函数，加签名，使得可SaveModel
  - ### 6.1 基本对象判断
  - ### 6.2 ConcreteFunction的图操作
    - `graph`
	- `graph.get_operations`
	- `graph.get_operation_by_name`
	- `graph.get_tensor_by_name`
	- `graph.as_graph_def`

## 1. 模块导入

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn

from tensorflow import keras
import tensorflow as tf
import sys
import os
import time
import datetime

for module in [np, pd, mpl, sklearn, keras, tf]:
    print(module.__name__, module.__version__)

numpy 1.18.1
pandas 0.25.3
matplotlib 3.1.2
sklearn 0.22.1
tensorflow_core.python.keras.api._v2.keras 2.2.4-tf
tensorflow 2.1.0


## 2. `tf.function`的用法

  - ### 2.1 函数式用法

In [3]:
# 普通函数
def scaled_elu(z, scale=2.0, alpha=1.0):
    is_positive = tf.greater_equal(z, 0.0)
    return scale * tf.where(is_positive, z, alpha*tf.nn.elu(z))

print(scaled_elu(tf.constant(-6.0)))
print(scaled_elu(tf.constant([3.0, 7.0])))

tf.Tensor(-1.9950424, shape=(), dtype=float32)
tf.Tensor([ 6. 14.], shape=(2,), dtype=float32)


In [5]:
# 使用tf.function 变成图
scaled_elu_tf = tf.function(scaled_elu)
print(scaled_elu_tf(tf.constant(-6.0)))
print(scaled_elu_tf(tf.constant([3.0, 7.0])))

# 还可以在变回去 
print(scaled_elu_tf.python_function is scaled_elu)

tf.Tensor(-1.9950424, shape=(), dtype=float32)
tf.Tensor([ 6. 14.], shape=(2,), dtype=float32)
True


>普通函数与加了tf.function的函数运算结果都一样，但是，tf.function的函数运算快，接下来，做测试

In [6]:
%timeit scaled_elu(tf.random.normal((10000, 10000)))
%timeit scaled_elu_tf(tf.random.normal((10000, 10000)))

18.9 ms ± 8.62 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
18.5 ms ± 6.35 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


  - ### 2.2 `@tf.function`装饰器的用法

In [7]:
# 1 + 1/2 + 1/2^2 + ....

@tf.function
def converage_to_2(n_iters):
    total = tf.constant(0.0)
    increment = tf.constant(1.0)
    for _ in range(n_iters):
        total += increment
        increment /= 2.0
    return total

print(converage_to_2(60))

tf.Tensor(2.0, shape=(), dtype=float32)


## 3. `tf.autograph` 的用法
[官方文档链接](https://tensorflow.google.cn/api_docs/python/tf/autograph)

  - ### 3.1 `tf.autograph.to_graph`,是`tf.function` 的低级 分装

In [10]:
scaled_elu_to_graph = tf.autograph.to_graph(scaled_elu)  # 是tf.function 的低级 分装
print(scaled_elu_to_graph(tf.constant([2.0, 5.0])))

print(scaled_elu_to_graph is scaled_elu_tf)

tf.Tensor([ 4. 10.], shape=(2,), dtype=float32)
False


  - ### 3.2 `tf.autograph.to_code`, 把函数变成图，再输出代码

In [11]:
def display_tf_code(func):
    code = tf.autograph.to_code(func)
    from IPython.display import display, Markdown
    display(Markdown("```python\n{}\n```".format(code)))

In [12]:
display_tf_code(scaled_elu)

```python
def tf__scaled_elu(z, scale=None, alpha=None):
  do_return = False
  retval_ = ag__.UndefinedReturnValue()
  with ag__.FunctionScope('scaled_elu', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
    is_positive = ag__.converted_call(tf.greater_equal, (z, 0.0), None, fscope)
    do_return = True
    retval_ = fscope.mark_return_value(scale * ag__.converted_call(tf.where, (is_positive, z, alpha * ag__.converted_call(tf.nn.elu, (z,), None, fscope)), None, fscope))
  do_return,
  return ag__.retval(retval_)

```

## 4. 变量不能包含在`tf.function`内

In [14]:
# 正常函数

def add_22():
    var = tf.Variable(1.0)
    var.assign_add(22)
    return var

print(add_22().value())

tf.Tensor(23.0, shape=(), dtype=float32)


In [16]:
# 加 tf.function 报错

@tf.function
def add_22():
    var = tf.Variable(1.0)
    var.assign_add(22)
    return var

try:
    print(add_22().value())
except Exception as e:
    print(e)

in converted code:

    <ipython-input-15-b7012a105b37>:3 add_22  *
        var = tf.Variable(1.0)
    /home/gaosy/environment/tf2_py3/lib/python3.6/site-packages/tensorflow_core/python/ops/variables.py:260 __call__
        return cls._variable_v2_call(*args, **kwargs)
    /home/gaosy/environment/tf2_py3/lib/python3.6/site-packages/tensorflow_core/python/ops/variables.py:254 _variable_v2_call
        shape=shape)
    /home/gaosy/environment/tf2_py3/lib/python3.6/site-packages/tensorflow_core/python/ops/variables.py:65 getter
        return captured_getter(captured_previous, **kwargs)
    /home/gaosy/environment/tf2_py3/lib/python3.6/site-packages/tensorflow_core/python/eager/def_function.py:502 invalid_creator_scope
        "tf.function-decorated function tried to create "

    ValueError: tf.function-decorated function tried to create variables on non-first call.



In [18]:
# 把变量放到外面，就不报错了

var = tf.Variable(1.0)
@tf.function
def add_22():
    var.assign_add(22)
    return var

print(add_22())  # 不用 value 方法 都能返回 Tensor

tf.Tensor(23.0, shape=(), dtype=float32)


## 5. `tf.function`签名 `input_signature`

In [20]:
@tf.function(input_signature=[tf.TensorSpec([None], tf.int32, name="x")])
def cube(x):
    return tf.pow(x, 3)

try:
    print(cube(tf.constant([1.0, 2.0]))) # 输入的数据类型，必须要和 签名的数据类型保持一致，否则报错
except Exception as e:
    print(e)

print(cube(tf.constant([1, 2])))

Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor([1. 2.], shape=(2,), dtype=float32))
  input_signature: (
    TensorSpec(shape=(None,), dtype=tf.int32, name='x'))
tf.Tensor([1 8], shape=(2,), dtype=int32)


In [22]:
# tf.function 传参 签名
def cube2(x):
    return tf.pow(x, 3)

cube2_tf = tf.function(cube2, input_signature=[tf.TensorSpec([None], tf.int32, name="x")])

try:
    print(cube2_tf(tf.constant([1.0, 2.0]))) # 输入的数据类型，必须要和 签名的数据类型保持一致，否则报错
except Exception as e:
    print(e)

print(cube2_tf(tf.constant([1, 2])))
cube2_tf

Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor([1. 2.], shape=(2,), dtype=float32))
  input_signature: (
    TensorSpec(shape=(None,), dtype=tf.int32, name='x'))
tf.Tensor([1 8], shape=(2,), dtype=int32)


## 6. `get_concrete_function`, 把变成图的函数，加签名，使得可SaveModel

  - ### 6.1 基本对象判断

In [29]:
cube2_tf = tf.function(cube2)  # 没有加签名

cube2_tf_signature = cube2_tf.get_concrete_function(tf.TensorSpec([None], tf.int32, name="x"))                                                 
print(cube2_tf_signature)      

print(cube2_tf_signature is cube2_tf.get_concrete_function(tf.TensorSpec([6], tf.int32, name="x"))) # 没有签名的都不一样

print(cube2_tf_signature is cube2_tf.get_concrete_function(tf.constant([1, 2, 3]))) # 没有签名的都不一样

False
False


In [30]:
cube2_tf = tf.function(cube2, input_signature=[tf.TensorSpec([None], tf.int32, name="x")]) # 有加签名

cube2_tf_signature = cube2_tf.get_concrete_function(tf.TensorSpec([None], tf.int32, name="x"))                                                 
print(cube2_tf_signature)      

print(cube2_tf_signature is cube2_tf.get_concrete_function(tf.TensorSpec([6], tf.int32, name="x"))) # 有签名的都一样

print(cube2_tf_signature is cube2_tf.get_concrete_function(tf.constant([1, 2, 3]))) # 有签名的都一样

True
True


  - ### 6.2 ConcreteFunction的图操作

In [31]:
cube2_tf_signature.graph

In [33]:
# 打印所有的操作
cube2_tf_signature.graph.get_operations()

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'Pow/y' type=Const>,
 <tf.Operation 'Pow' type=Pow>,
 <tf.Operation 'Identity' type=Identity>]

In [34]:
# 看看某一个操作
pow_op = cube2_tf_signature.graph.get_operations()[2]
print(pow_op)

name: "Pow"
op: "Pow"
input: "x"
input: "Pow/y"
attr {
  key: "T"
  value {
    type: DT_INT32
  }
}



In [35]:
# 看看该操作的输入输出
print(list(pow_op.inputs))
print(list(pow_op.outputs))

[<tf.Tensor 'x:0' shape=(None,) dtype=int32>, <tf.Tensor 'Pow/y:0' shape=() dtype=int32>]
[<tf.Tensor 'Pow:0' shape=(None,) dtype=int32>]


In [36]:
# 获取某一个操作
cube2_tf_signature.graph.get_operation_by_name("x")

<tf.Operation 'x' type=Placeholder>

In [37]:
# 获取某一个Tensor
cube2_tf_signature.graph.get_tensor_by_name("x:0")

<tf.Tensor 'x:0' shape=(None,) dtype=int32>

In [38]:
# 打印图结构
cube2_tf_signature.graph.as_graph_def()

node {
  name: "x"
  op: "Placeholder"
  attr {
    key: "_user_specified_name"
    value {
      s: "x"
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
node {
  name: "Pow/y"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 3
      }
    }
  }
}
node {
  name: "Pow"
  op: "Pow"
  input: "x"
  input: "Pow/y"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
node {
  name: "Identity"
  op: "Identity"
  input: "Pow"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
versions {
  producer: 175
}